https://www.acmicpc.net/step/26

## 최단경로
방향그래프가 주어지면 주어진 시작점에서 다른 모든 정점으로의 최단 경로를 구하는 프로그램을 작성하시오. (모든 간선의 가중치는 10 이하의 자연수)

<img src="./img/최단경로1.jpeg" width=400 />


1. 정점 1에서 직접 갈 수 있는 정점은 2, 3이고, 문제에서 시작점 자신은 0으로 출력하라고 했으니 테이블은


    | |1|2|3|4|5|
    |-|-|-|-|-|-|
    |start(1)|0|2|3|inf|inf|

    <br></br>

2. 자신을 제외한 가장 가까운 정점을 선택하여 테이블 갱신하는 알고리즘 수행
    1. 현재 테이블에서 가장 가까운 정점 2 선택
    2. 정점 2에서 갈 수 있는 정점 조사
        1. 2 -> 3) 4, 2 -> 4) 5 데이터를 최소 힙에 넣는다.
        2. 최소 힙에서 가장 가중치가 적은 노드인 2 -> 3) 4 를 선택한다.

        1 -> 2 -> 3) 2+4=6

        현재 테이블에서 정점 3까지 거리는 3으로 1 -> 3 경우가 6보다 더 작으므로 테이블 업데이트 없이 다음 단계 진행
        3. 최소 힙에서 가장 가중치가 작은 노드를 선택한다. 2 -> 4) 5 선택

        1 -> 2 -> 4) 2+5=7

        현재 테이블에서 정점 4까지 거리는 inf이므로 7로 업데이트 한다.
        
        | |1|2|3|4|5|
        |-|-|-|-|-|-|
        |start(2)|0|2|3|7|inf|

        <br></br>

3. 1-2 과정을 최소 힙이 비어질 때까지 반복한다.

In [5]:
import sys 
import heapq 

INF = 10000000
V, E = map(int, input().split()) 
K = int(input()) #시작점 K 
dp = [INF]*(V+1) #가중치 테이블 dp 
heap = [] 
graph = [[] for _ in range(V + 1)]

for _ in range(E):
    u, v, w = map(int, input().split())
    graph[u].append([v, w])

print(V, E, K)
print(graph)

5 6 1
[[], [[2, 2], [3, 3]], [[3, 4], [4, 5]], [[4, 6]], [], [[1, 1]]]


In [7]:
def dijkstra(start):
    dp[start] = 0 # 가중치 테이블에서 시작 정점은 0으로 초기화하고 시작
    heapq.heappush(heap, [0, start]) # 맨 첫 정점 정보 넣어줌

    while heap: # 힙에 원소가 남아있으면 반복
        w, n = heapq.heappop(heap)
        for n_n, wei in graph[n]:
            n_w = wei + w     # 현재 정점까지의 가중치 + 현재 정점에서 다음 정점까지의 가중치가
            if n_w < dp[n_n]: # 현재 기록된 깂보다 작으면
                dp[n_n] = n_w # 가중치 테이블 없데이트
                heapq.heappush(heap, [n_w, n_n]) # [다음 점까지의 가중치, 다음 점] 최소 힙에 삽입

dijkstra(K)
for i in dp[1:]:
    print(i if i != INF else "INF")

0
2
3
7
INF


## 특정한 최단 경로
방향성이 없는 그래프가 주어진다. 정점 1에서 정점 N까지 최단 거리로 이동하려고 한다.

조건
1. 임의로 주어진 두 정점을 반드시 통과해야 한다.
2. 한번 이동했던 정점, 한번 이동했던 간선 모두 다시 이동할 수 있다. (하지만 반드시 최단 경로로 이동해야 함!)

<img src="./img/특정한 최단경로.jpeg" width=450 />

정점 1 -> 정점 N 까지 이동하는 경로는 두 가지로 나눌 수 있다.
-   1 - v1 - v2 - vn
-   1 - v2 - v1 - vn

따라서 3번의 다익스트라 함수를 통해 구할 수 있다.
- 1에서 출발해서 1 ~ n 의 최단 경로를 구한다. 

    -> 1 ~ v1, 1 ~ v2 의 길이를 구할 수 있다.
- v1에서 출발해서 v1 ~ n 의 최단 경로를 구한다.

    -> v1 ~ n 의 길이를 구할 수 있다.
- v2에서 출발해서 v2 ~ n 의 최단 경로를 구한다.

    -> v2 ~ n 의 길이를 구할 수 있다.  

```
from_1 = dijkstra(1); from_v1 = dijkstra(v1); from_v2 = dijkstra(v2) 
path1 = from_1[v1] + from_v1[v2] + from_v2[n] 
path2 = from_1[v2] + from_v2[v1] + from_v1[n]
```

두 경로 중 최소 경로를 선택한다.

In [11]:
import collections, heapq

n, e = map(int, input().split()) 
g = collections.defaultdict(list) ; print("defaultdict: ", g)
for _ in range(e): 
    a, b, c = map(int, input().split()) 
    g[a].append([b, c]) 
    g[b].append([a, c]) 

v1, v2 = map(int, input().split()) 
print(g)

defaultdict:  defaultdict(<class 'list'>, {})
defaultdict(<class 'list'>, {1: [[2, 3], [3, 5], [4, 4]], 2: [[1, 3], [3, 3], [4, 5]], 3: [[2, 3], [4, 1], [1, 5]], 4: [[3, 1], [2, 5], [1, 4]]})


In [12]:
INF = 1000000

def dijkstra(s): 
    dist = [INF] * (n+1) 
    dist[s] = 0 
    q = [[dist[s], s]] 
    while q: 
        u = heapq.heappop(q)[1] 
        for v, c in g[u]: 
            if dist[v] > dist[u] + c: 
                dist[v] = dist[u] + c 
                heapq.heappush(q, [dist[v], v]) 
    return dist 
    
def solve(): 
    from_1 = dijkstra(1) 
    from_v1 = dijkstra(v1) 
    from_v2 = dijkstra(v2) 
    
    path1 = from_1[v1] + from_v1[v2] + from_v2[n] 
    path2 = from_1[v2] + from_v2[v1] + from_v1[n] 
    
    result = min(path1, path2) 
    
    if result < INF: 
        return result 
    else: 
        return -1 
        

print(solve())

7


## 미확인 도착지
쒸익쒸익..... 서커스 듀오가 도시의 거리들을 이동히는데, 얘네가 어디로 가고 있는지 알아내야 한다.

- 듀오는 s 지점에서 출발했다.
- 목적지 후보들 중 하나가 듀오의 목적지다.
- 듀오는 목적지까지 최단거리로 간다.
- 듀오는 g와 h 교차로 사이에 있는 도로를 지나갔다.

듀오의 목적지를 구하는 프로그램을 작성하시오.

<img src="./img/미확인도착지.jpeg" width=450 />

### 풀이 key
출발점 s에서 각 목적지 후보 t까지 가는 최단 경로 중 g, h 정점을 경유하는 경우를 구한다.

이를 보장하기 위해서 만족해야 하는 조건은
- s->t 최단경로 = s->g 최단경로 + g->h 최단경로 + h->t 최단경로
- s->t 최단경로 = s->h 최단경로 + h->g 최단경로 + g->t 최단경로
    - s->t 최단경로 : s 기준으로 다익스트라
    - g->h 최단경로 : 문제 조건에서 주어짐
    - g(h)->t 최단경로 : g(h) 에 대해 다익스트라

In [14]:
import heapq as hq
n_test = int(input())

def Dijistra(s): #s를 기준으로 하는 다익스트라
    q=[]
    dist =[float('inf') for i in range(n+1)]
    dist[s] =0
    hq.heappush(q,(0,s))

    while q:
        weight, vertice= hq.heappop(q)
        if dist[vertice] < weight:
            continue

        for w, ver in adj[vertice]:
            if dist[ver] > w + weight: #현재 거리가 더 짧은 경로라면
                dist[ver]= w+weight
                hq.heappush(q, (w + weight, ver))

    return dist #거리 리스트를 반환한다.


for i in range(n_test):
    n, m, T = map(int, input().split())  # n교차로 m은 도로  t 목적지 후보 갯
    adj = [[] for i in range(n + 1)] #1번부터 n번노드까지 사용
    s, g, h = map(int, input().split())
    de=[]
    # s 시작점 g와 h 사이의 교차로 사이에 있는 도로를 지나갔다.
    # 목적지 후보중  g와 h를 경유하는 최단 경로
    # 최단경로가 g,h를 포함하는 목적지를 찾으면된다.
    # 최단경로가 여러개 있을 수 있다.

    for i in range(m):
        a, b, d = map(int, input().split())
        adj[a].append((d, b))
        adj[b].append((d, a))
        if (a== g and b== h) or (a==h and b==g):
            path_g_h =d
    for i in range(T):
        de.append(int(input()))



    dist_s = Dijistra(s) #시작점에서 모든 노드로의 거리
    dist_h = Dijistra(h) #h에서 모든 노드로의 거리
    dist_g = Dijistra(g) #g에서 모든 노드로의 거리

    path_s_g = dist_s[g] # s에서 g로 가는 최단거리
    path_s_h = dist_s[h] # s에서 h로 가는 최단거리

    answer =[]
    for t in de:
        path_g_t = dist_g[t] #g에서 t로 가는 최단거리
        path_h_t = dist_h[t] #h에서 t로 가는 최단거리
        path_s_t = dist_s[t] #s에서 t로 가는 최단거리
        
        if path_g_t ==float('inf') or path_h_t ==float('inf') or path_s_t ==float('inf'):
            continue
            # 거리값들중 무한대 값이 있으면 연결이 안되있는 노드임으로 불가능

        path_s_g_h_t = path_s_g + path_g_h + path_h_t
        path_s_h_g_t = path_s_h + path_g_h + path_g_t
        

        if (path_s_g_h_t == path_s_t or  path_s_h_g_t == path_s_t):
            answer.append(t)

    answer.sort()

    print(' '.join(map(str, answer)))

4 5
6


## 타임머신
N개의 도시가 있고 각 도시를 연결하는 버스가 M개 있다.

각 버스는 a(시작 도시), b(도착 도시), c(버스를 타고 이동하는데 걸리는 시간) 으로 나타낼 수 있다.

시간 c는 양수가 아닐 수 있다. c=0의 경우 순간 이동을 하는 경우고, c<0의 경우 타임머신으로 시간을 되돌아가는 경우다.

1번 도시에서 출발해서 나머지 도시로 가는 가장 빠른 시간을 구하는 프로그램을 작성하시오.

 -> 음수 간선이 있는 경우이므로 벨먼포드 알고리즘 이용

In [16]:
import collections 

INF = float('inf') 
g = collections.defaultdict(list) 
n, m = map(int, input().split()) 
for _ in range(m): 
    a, b, c = map(int, input().split()) 
    g[a].append([b, c]) 

def bellman_ford(s): 
    dist = [INF] * (n+1) 
    dist[s] = 0 
    for _ in range(n-1): 
        for u in range(1, n+1): 
            for v, c in g[u]: 
                if dist[v] > dist[u] + c: 
                    dist[v] = dist[u] + c 
    for u in range(1, n+1): 
        for v, c in g[u]: 
            if dist[v] > dist[u] + c: 
                return False 
                
    return dist 
    
dist = bellman_ford(1) 

if dist == False: 
    print(-1) 
else: 
    for i in range(2, n+1): 
        print(dist[i] if dist[i] < INF else -1)

4
3
